In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import os
os.chdir("/content/drive/My Drive")

In [3]:
!pip install segmentation_models
!pip install tensorflow==2.2.0
!pip install keras==2.3.1

     |████████████████████████████████| 51kB 2.9MB/s 
     |████████████████████████████████| 516.2MB 29kB/s 
     |████████████████████████████████| 460kB 43.8MB/s 
     |████████████████████████████████| 3.0MB 42.4MB/s 
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tensorflow-estimator-2.3.0
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing installation: tensorflow 2.3.0
    Uninstalling tensorflow-2.3.0:
      Successfully uninstalled tensorflow-2.3.0
     |████████████████████████████████| 378kB 3.4MB/s 
  Found existing installation: Keras 2.4.3
    Uninstalling Keras-2.4.3:
      Successfully uninstalled Keras-2.4.3


In [4]:
import os
import cv2
import segmentation_models as sm
import tensorflow as tf
import keras.utils
import numpy as np
from PIL import Image
from keras.optimizers import adam
from keras import backend as K
MAX_EPOCHS = 20
image_size = 256
BACKBONE = 'resnet34'
preprocess_input = sm.get_preprocessing(BACKBONE)

Using TensorFlow backend.


Segmentation Models: using `keras` framework.


In [5]:
class Data_Gen(keras.utils.Sequence):

  def __init__(self, batch_size, x_set, y_set1, y_set2,classes = 2):
      self.x, self.y1, self.y2 = np.array(x_set), np.array(y_set1), np.array(y_set2)
      self.batch_size = batch_size
      #self.indices=np.arange(len(x_set))
      self.indices = np.arange(self.x.shape[0])
      np.random.shuffle(self.indices)
      #self.type = type
      #self.preproc = preproc

  def __len__(self):
      # print(self.type + ' - len : ' + str(int(np.ceil(self.x.shape[0] / self.batch_size))))
      return int(np.ceil(self.x.shape[0] / self.batch_size))

  def __getitem__(self, idx):
      #inds = self.indices[idx * self.batch_size:(idx + 1) * self.batch_size]
      upper_bound = len(self.indices) if ((idx + 1) * self.batch_size) > len(self.indices)  else (idx + 1) * self.batch_size    
      inds = self.indices[idx * self.batch_size:upper_bound]
      batch_x = np.empty((len(inds),image_size,image_size,3))  
      batch_y = np.empty((len(inds),image_size,image_size,2))
      for i in range(len(inds)):
        image_x = cv2.imread(self.x[inds[i]]) 
        imx = cv2.resize(image_x, (image_size,image_size), interpolation=cv2.INTER_CUBIC)
        imagex = np.zeros(imx.shape,dtype = np.float32)
        cv2.normalize(imx,imagex,alpha = 0,beta = 1,norm_type = cv2.NORM_MINMAX,dtype = cv2.CV_32F)  
        batch_x[i] = cv2.cvtColor(imagex,cv2.COLOR_BGR2RGB)  
        
        disc_y = cv2.imread(self.y1[inds[i]],cv2.IMREAD_GRAYSCALE)/255 
        dcy = cv2.resize(disc_y, (image_size,image_size))
        #dcy = np.expand_dims(dcy, axis=0)
        batch_y[i,:,:,0] = dcy
        
        cup_y = cv2.imread(self.y2[inds[i]],cv2.IMREAD_GRAYSCALE)/255 
        cpy = cv2.resize(cup_y, (image_size,image_size))
        #cpy = np.expand_dims(cpy, axis=1)
        batch_y[i,:,:,1] = cpy
        
      return batch_x, batch_y

  def on_epoch_end(self):
      np.random.shuffle(self.indices)

In [6]:
batch_size = 10

images_dir = os.path.join('./4data/Training/', 'image for cup')
ids = os.listdir(images_dir)
ids.sort()
images_fps  = [os.path.join(images_dir, image_id) for image_id in ids]
print(images_fps)
disc_dir = os.path.join('./4data/Training/', 'mask_disc2')
iddisc = os.listdir(disc_dir)
iddisc.sort()
disc_fps = [os.path.join(disc_dir, disc_id) for disc_id in iddisc]
print(disc_fps)
cup_dir = os.path.join('./4data/Training/', 'mask_cup')
idcup = os.listdir(cup_dir)
idcup.sort()
cup_fps = [os.path.join(cup_dir, cup_id) for cup_id in idcup]
print(cup_fps)



test_images_dir = os.path.join('./4data/Validation/', 'image for cup')
ids = os.listdir(test_images_dir)
ids.sort()
test_images_fps  = [os.path.join(test_images_dir, image_id) for image_id in ids]
print(test_images_fps)
test_disc_dir = os.path.join('./4data/Validation/', 'mask_disc2')
iddisc = os.listdir(test_disc_dir)
iddisc.sort()
test_disc_fps = [os.path.join(test_disc_dir, disc_id) for disc_id in iddisc]
print(test_disc_fps)
test_cup_dir = os.path.join('./4data/Validation/', 'mask_cup')
idcup = os.listdir(test_cup_dir)
idcup.sort()
test_cup_fps = [os.path.join(test_cup_dir, cup_id) for cup_id in idcup]
print(test_cup_fps)


train_gen = Data_Gen(batch_size, images_fps, disc_fps, cup_fps,classes=2)
val_gen = Data_Gen(batch_size, test_images_fps,test_disc_fps,test_cup_fps,classes=2)

['./4data/Training/image for cup/T0001.jpg', './4data/Training/image for cup/T0002.jpg', './4data/Training/image for cup/T0003.jpg', './4data/Training/image for cup/T0004.jpg', './4data/Training/image for cup/T0005.jpg', './4data/Training/image for cup/T0006.jpg', './4data/Training/image for cup/T0007.jpg', './4data/Training/image for cup/T0008.jpg', './4data/Training/image for cup/T0009.jpg', './4data/Training/image for cup/T0010.jpg', './4data/Training/image for cup/T0011.jpg', './4data/Training/image for cup/T0012.jpg', './4data/Training/image for cup/T0013.jpg', './4data/Training/image for cup/T0014.jpg', './4data/Training/image for cup/T0015.jpg', './4data/Training/image for cup/T0016.jpg', './4data/Training/image for cup/T0017.jpg', './4data/Training/image for cup/T0018.jpg', './4data/Training/image for cup/T0019.jpg', './4data/Training/image for cup/T0020.jpg', './4data/Training/image for cup/T0021.jpg', './4data/Training/image for cup/T0022.jpg', './4data/Training/image for cup

In [7]:
from segmentation_models import Unet
from segmentation_models import get_preprocessing

#import sys
#sys.path.append('/content/drive/My Drive/segmentation_models/segmentation_models')

In [28]:
EPS = 1e-7
def vertical_diameter(binary_segmentation):

 
    # turn the variable to boolean, just in case
    binary_segmentation = np.asarray(binary_segmentation, dtype=np.bool)
   
    # get the sum of the pixels in the vertical axis
    vertical_axis_diameter = np.sum(binary_segmentation, axis=0)

    # pick the maximum value
    diameter = np.max(vertical_axis_diameter)

    # return it
    return float(diameter)



def vertical_cup_to_disc_ratio(segmentation):
    # compute the cup diameter
    cup_diameter = vertical_diameter(segmentation[:, :, 1]==0)
    # compute the disc diameter
    disc_diameter = vertical_diameter(segmentation[ :, :, 0]<255)

    return cup_diameter / (disc_diameter + EPS)

def absolute_error(predicted, reference):
    return abs(predicted - reference)

def CDR(y_true, y_pred):

    segmentation = y_pred
    gt_label = y_true
    print(y_true.shape)
    for i in range(len(y_true)):
      cdr[i] = absolute_error(vertical_cup_to_disc_ratio(segmentation[i]), vertical_cup_to_disc_ratio(binary_gt_label[i]))    
    print(cdr.shape)
    return cdr


def BCE(y_true, y_pred):
  y_true_disc = y_true[:, :, :, 0]
  y_true_cup = y_true[:,:,:, 1]
  y_pred_disc = y_pred[:, :, :, 0]
  y_pred_disc = K.cast(y_pred_disc > 0.5, 'float32')
  y_pred_cup = y_pred[:,:,:, 1]
  y_pred_cup = K.cast(y_pred_cup > 0.5, 'float32')
  print(K.binary_crossentropy(y_true, y_pred).shape)
  return K.binary_crossentropy(y_true, y_pred)

def customLoss(y_true, y_pred):
  loss =  BCE(y_true, y_pred)+CDR(y_true, y_pred)
  return loss

In [29]:
from segmentation_models import losses
from segmentation_models import metrics


opt = adam(lr = 0.0001)
model = Unet(BACKBONE,input_shape=(256, 256, 3), encoder_weights='imagenet', classes = 2)
model.compile(optimizer = opt, loss= customLoss) 
#model.summary()

(None, 256, 256, 2)
(None, None, None, None)


TypeError: ignored

In [ ]:
 model_checkpoint = keras.callbacks.ModelCheckpoint('./segWeight_disc/weight_multiSeg_unet_res.hdf5', 
                                                   monitor="val_loss", 
                                                   mode="min", verbose=1, 
                                                   save_best_only=True)
history = model.fit_generator(train_gen,
                              epochs=MAX_EPOCHS,
                              validation_data=val_gen,
                              callbacks=[model_checkpoint])